# Développement d’un module de classification d’images IHC par IA

### By Maëlle Broustal, Rafik Mankour & Ian Pellet

## Import

In [ ]:
'''
!apt update && apt install -y openslide-tools
!pip install openslide-python
'''

In [ ]:
from xml.dom import minidom
'''
# Images
import os
import PIL 
import numpy as np
import openslide

# Annotations
from xml.dom import minidom
import skimage.draw

#import pandas as pd
#import skimage
#import matplotlib.pyplot as plt
#from tensorflow import keras as k
'''

In [ ]:
import pipeline_IHC as pipe

## Training

### Chargement des images et annotations

Charger les images et les fichiers annotation et les transformer en array

#### Images

In [ ]:
input_dir = "./data/"
input_img = pipe.load_img(input_dir)

In [ ]:
list(input_img.keys())

#### Annotations

In [ ]:
loaded_masks = pipe.load_annot(input_dir, input_img)

In [ ]:
len(loaded_masks) == len(input_img)

In [ ]:
ok = True
for img in loaded_masks:
    if loaded_masks[img].shape != input_img[img].shape:
        ok = False
        break
ok

In [ ]:
annot = minidom.parse("./test_data/data/test.annotations") # open XML file
Vcoords = pipe.xml_to_vertices(annot)

In [ ]:
Vcoords

In [ ]:
mask = pipe.vertices_to_mask((1008, 840), 64, Vcoords)

In [ ]:
len(mask[0])

## Mise en place du réseau de neurones

Adapter ce code issu des liens dans le sprint backlog avec des données :
Problème à régler -> faire correspondre la taille et la dimension de l'image ) celle de la matrice d'annotation

In [ ]:
all_images = []
all_images.append(full_np)

all_annotations = []
all_annotations.append(img)

x_train = np.array(all_images)
y_train = np.array(all_annotations)
y_train = k.utils.to_categorical(y_train, 2)

print(np.shape(x_train))
np.shape(y_train)

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(200, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# train the model on the new data for a few epochs

model.fit(x_train, y_train, batch_size=1, epochs=10, verbose=1)

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers

model.fit(x_train, y_train, batch_size=1, epochs=10, verbose=1)

# FOURNIR X_TRAIN AVEC LES IMAGES et Y_TRAIN AVEC LES ANNOTATIONS A EMPLOYER

for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers

## Test du réseau de neurones